In [4]:
!pip install keras-tuner

import tensorflow as tf
from tensorflow import keras
import numpy as np

fashion_mnist=keras.datasets.fashion_mnist

(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

train_images=train_images/255.0
test_images=test_images/255.0

train_images[0].shape

train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)


def build_model(hp):
   model = keras.Sequential([
                             keras.layers.Conv2D(
                                 filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
                                 kernel_size=hp.Choice('conv_1_kernal',values = [3,5]),
                                 activation='relu',
                                 input_shape=(28,28,1)
                             ),
                              keras.layers.Conv2D(
                                  filters=hp.Int('conv_2__filter', min_value=32, max_value=64, step=16),
                                  kernel_size=hp.Choice('conv_2_kernal',values = [3,5]),
                                   activation='relu'
                              ),
                             keras.layers.Flatten(),
                             keras.layers.Dense(
                                  units=hp.Int('Dense_1_units', min_value=32, max_value=128, step=16),
                                  activation='relu'
                             ),
                              keras.layers.Dense(10, activation='softmax')#output layer
   ])
   model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2, 1e-3])),
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
   return model

from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

tuner_search=RandomSearch(build_model,
                              objective='val_accuracy',
                              max_trials=5,directory='output',project_name="Mnist Fashion")
    
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

model=tuner_search.get_best_models(num_models=1)[0]

model.summary()
                         

Trial 4 Complete [00h 26m 22s]
val_accuracy: 0.8983333110809326

Best val_accuracy So Far: 0.9153333306312561
Total elapsed time: 00h 49m 06s
INFO:tensorflow:Oracle triggered exit
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        51264     
_________________________________________________________________
flatten (Flatten)            (None, 30976)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                2973792   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 3,026,346
Trainable params: 3,026,346
Non-trainable params: 